In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from tabulate import tabulate
import matplotlib.pyplot as plt
%matplotlib inline

## Data

#### Generate FASTA files

In [ ]:
%%bash
awk -F "\t" '{print ">train_"NR"_score_"$2"\n"$1}' data/train_sequences.txt > data/train.fasta
awk -F "\t" '{print ">test_"NR"_score_"$2"\n"$1}' data/test_sequences.txt > data/test.fasta

In [ ]:
! head -n 4 data/train.fasta

#### Load tabulated data

In [ ]:
train = pd.read_csv('data/train_sequences.txt', sep='\t', header=None, names=["sequence", "score"])
test = pd.read_csv('data/test_sequences.txt', sep='\t', header=None, names=["sequence", "score"])

#### Mini dataset

In [ ]:
%%bash
head -n 1024 data/train_sequences.txt > data/train_sequences_mini.txt
head -n 256 data/test_sequences.txt > data/test_sequences_mini.txt

## Sequence Exploration

In [ ]:
train.shape

In [ ]:
train["sequence"].apply(len).value_counts()

In [ ]:
train["sequence"].apply(len).hist(bins=50)

In [ ]:
test["sequence"].apply(len).value_counts()

Test seqs are all 110 length while training seq length varies.

In [ ]:
plt.scatter(train["sequence"].apply(len), train["score"], s=0.5, alpha=0.5)
plt.show()

No clear correlatoin between length and score.

### Dive into Sequence

Topics to explore:
* Sequence cluster using MMseqs2.
* overlapping 5-prime and 3-prime regions.
* If and how reverse complimentary of the true promoter region affects the score.

Sequence clustering was executed in cmd `cluster_sequences.sh`.

In [ ]:
# # document the excecution
%%bash
cut -c 1-17 data/train_sequences.txt | sort -u
awk -F "\t" '{l = length($1); print substr($1, l-12)}' data/train_sequences.txt | sort -u

All sequences have same 5-prime and 3-prime sequence. For train, there is 17 bp 5-prime flanking and 12 bp at 3-prime.

In [ ]:
%%bash
cd data/cluster
echo $(cut -f 1 train.cluster30_cluster.tsv | uniq | wc -l)
echo $(wc -l train.cluster30_cluster.tsv)

At 30% identity cutoff we can cluster the 6.7M sequences (not stripped) into 1.4M clusters.

In [ ]:
%%bash
cd data/cluster_stripped
echo $(cut -f 1 train.stripped.cluster30_cluster.tsv | uniq | wc -l)
echo $(wc -l train.stripped.cluster30_cluster.tsv)

If stripped ~95% sequences are singletons at 30% identity cutoff. This means the nearly all sequence identity comes from the 5-prime and 3-prime flanking regions. It also means *data leakage from homology* is not a big deal in this case.

## Score Exploration

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
ax.hist(train["score"].values, bins=18)
plt.show()

In [ ]:
report = tabulate([
    ('Max', train["score"].values.max()),
    ('Avg', train["score"].values.mean()),
    ('Med', np.median(train["score"].values)),
    ('Min', train["score"].values.min())
], tablefmt='fancy_grid'
)
print(report)

## Pre-processing Exploration

### Embedding
(A, T, C , G) + N

quote: Each promoter sequence is comprised of the bases A, T, G, and C, and rarely includes an N (in the training data), when a base could not be confidently called during DNA sequencing.

In [ ]:
# Passed! don't run again

# for s in tqdm(train[0].values):
#     for _ in s:
#         if _ not in ('A', 'T', 'C', 'G', 'N'):
#             print(s)

### Preprocess
use mini set as example

In [ ]:
import torch
import torch.nn as nn

In [ ]:
train = pd.read_csv('data/train_sequences_mini.txt', sep='\t', header=None, names=["sequence", "score"])
test = pd.read_csv('data/test_sequences_mini.txt', sep='\t', header=None, names=["sequence", "score"])

In [ ]:
# max length = 142 in train and length = 110 for all test sequences
maxlen = 150

##### pad sequence

In [ ]:
pad_dict = {
    'A': 0,
    'T': 1,
    'C': 2,
    'G': 3,
    'N': 4,
    '<PAD>': 5,
}

In [ ]:
def seq2tensor(seq, maxlength=150):
    tensor = torch.zeros(maxlength, dtype=torch.long) + pad_dict['<PAD>']
    for i in range(len(seq)):
        tensor[i] = pad_dict[seq[i]]
    return tensor

In [ ]:
def get_data(df):
    seqs = [None] * len(df)
    scores = [None] * len(df)
    for i in tqdm(range(len(df))):
        seqs[i] = seq2tensor(df["sequence"][i]).long()
        scores[i] = df["score"][i]
    seqs = torch.stack(seqs)
    scores = torch.tensor(scores)
    return seqs, scores

In [ ]:
train_seqs, train_scores = get_data(train)

In [ ]:
test_seqs, test_scores = get_data(test)

In [ ]:
torch.save((train_seqs, train_scores), 'train_mini.pt')

In [ ]:
torch.save((test_seqs, test_scores), 'test_mini.pt')

In [ ]:
train_seqs